# Encrypt the model

## Confirm the model file was dowloaded successfully

The original llm guide is available here: <https://github.com/rh-aiservices-bu/test-drive.git>. The step by step guide is <https://rh-aiservices-bu.github.io/rhoai-rh1-testdrive/modules/llm/index.html>.

In [16]:
%%bash
MODEL_NAME=flan-t5-small
echo "export MODEL_NAME=$MODEL_NAME" > .model

In [ ]:
%%bash
source .model

if [ ! -d $MODEL_NAME ]; then
    echo "Model does not exist! Downloading it again..."
    git clone https://huggingface.co/google/$MODEL_NAME
else # Saves a bit of time since the operation takes time
    echo "Model has been already downloaded"
fi

## Compress the model folder and encrypt it

Compress the model folder

In [ ]:
%%bash
source .model
MODEL_NAME_TAR=$MODEL_NAME.tar.gz
echo "export MODEL_NAME_TAR=$MODEL_NAME_TAR" >> .model

echo "Zipping the model, will take a while..."
if [ ! -f $MODEL_NAME_TAR ]; then
    cd $MODEL_NAME 
    rm -rf .git
    cd - > /dev/null
    tar czvf $MODEL_NAME_TAR $MODEL_NAME
else # Saves a bit of time since the operation takes time
    echo "Model has been already zipped"
fi
echo "Done!"



## Encrypt the model

The key used in this example comes from https://people.redhat.com/eesposit/key.bin, but of course you can use any other key. You can also generate a key with `openssl rand 128 > key.bin`. **This key has to also be loaded into Trustee.**

In [ ]:
! curl -L https://people.redhat.com/eesposit/key.bin -o key.bin

In [ ]:
%%bash
source .model

MODEL_NAME_ENC=$MODEL_NAME_TAR.enc
echo "export MODEL_NAME_ENC=$MODEL_NAME_ENC" >> .model

openssl enc -aes-256-cbc -pbkdf2 -kfile key.bin -in $MODEL_NAME_TAR -out $MODEL_NAME_ENC
echo "Encryption successful"

ls -l $MODEL_NAME_ENC

Move the encrypted file into the model folder

In [15]:
%%bash
source .model

MODEL_NAME_ENC_FOLDER=${MODEL_NAME}-enc
echo "export MODEL_NAME_ENC_FOLDER=$MODEL_NAME_ENC_FOLDER" >> .model

mkdir -p $MODEL_NAME_ENC_FOLDER && mv $MODEL_NAME_ENC $MODEL_NAME_ENC_FOLDER

Delete unnecessary files

In [19]:
%%bash
source .model

rm -rf $MODEL_NAME_TAR
rm -rf $MODEL_NAME
rm -rf key.bin

## Helper functions for upload

Instead of copying code, try to load the same function inside the original workshop, which should be located at `../../test-drive/llm/1_download_save.ipynb`

In [ ]:
! pip install nbimporter

In [ ]:
import nbformat

notebook_path = "../../test-drive/llm/1_download_save.ipynb"

with open(notebook_path) as f:
    nb = nbformat.read(f, as_version=4)

# Run cell 7 in 1_download_save.ipynb, ie the one under 
# "Helper functions for upload"
print(nb['cells'][6]['source'])
exec(nb['cells'][6]['source'])

## Check the Storage Bucket

In your S3 bucket, under the `models` upload prefix, run the `list_object` command. As best practice, to avoid mixing up model files, keep only one model and its required files in a given prefix or directory. This practice allows you to download and serve a directory with all the files that a model requires. 

If this is the first time running the code, this cell will have no output or the fraud model from the predictive AI/ML exercise.


In [ ]:
list_objects("models")

## Upload and check again

Use the function to upload the `models` folder in a rescursive fashion:

In [ ]:
upload_directory_to_s3("flan-t5-small-enc", "models/flan-t5-small")

To confirm this worked, run the `list_objects` function again:

This time, you should see the encrypted model listed in the directory/prefix: `models/flan-t5-small`

```
models/flan-t5-small/flan-t5-small.tar.gz.enc
```

In [ ]:
list_objects("models")